### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [1]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Carga de datos

In [2]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [3]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn
tfidfvect = TfidfVectorizer()

In [4]:
# en el atributo `data` accedemos al texto
newsgroups_train.data[0]

'I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.'

In [5]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [6]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'cantidad de documentos: {X_train.shape[0]}')
print(f'tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
cantidad de documentos: 11314
tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [7]:
# una vez fiteado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25775

In [8]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [9]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [10]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [11]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 4811
print(newsgroups_train.data[idx])

THE WHITE HOUSE

                  Office of the Press Secretary
                   (Pittsburgh, Pennslyvania)
______________________________________________________________
For Immediate Release                         April 17, 1993     

             
                  RADIO ADDRESS TO THE NATION 
                        BY THE PRESIDENT
             
                Pittsburgh International Airport
                    Pittsburgh, Pennsylvania
             
             
10:06 A.M. EDT
             
             
             THE PRESIDENT:  Good morning.  My voice is coming to
you this morning through the facilities of the oldest radio
station in America, KDKA in Pittsburgh.  I'm visiting the city to
meet personally with citizens here to discuss my plans for jobs,
health care and the economy.  But I wanted first to do my weekly
broadcast with the American people. 
             
             I'm told this station first broadcast in 1920 when
it reported that year's presidential elec

In [12]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [13]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.70930477, 0.67474953, ..., 0.        , 0.        ,
       0.        ])

In [14]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([ 4811,  6635,  4253, ...,  1534, 10055,  4750])

In [15]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [16]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'talk.politics.misc'

In [17]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc


### Modelo de clasificación Naïve Bayes

In [18]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [19]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [20]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

### Consigna del desafío 1

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.


### 1.

In [ ]:
import random

# Seleccionamos 5 documentos al azar
random_docs = random.sample(range(len(newsgroups_train.data)), 5)

for doc_idx in random_docs:
    print(f"\nDocumento seleccionado (índice {doc_idx}):")
    # Mostramos los primeros 500 caracteres
    print(newsgroups_train.data[doc_idx][:500], '...')

    # Similaridad con todos los documentos de train
    cossim = cosine_similarity(X_train[doc_idx], X_train)[0]

    # Obtenemos los 5 documentos más similares (excluyendo el documento en sí mismo)
    mostsim = np.argsort(cossim)[::-1][1:6]

    print("\nClases de los 5 documentos más similares:")
    for i in mostsim:
        print(f"Índice {i}: {newsgroups_train.target_names[y_train[i]]}")



Documento seleccionado (índice 5929):
Sorry but I forgot this ps.

Right now my sight is getting news about two weeks behind so it's 
kind of necessary (to me) that any responses be sent to me directly.


Thanks a lot ...

Clases de los 5 documentos más similares:
Índice 3166: soc.religion.christian
Índice 7532: soc.religion.christian
Índice 8628: comp.sys.ibm.pc.hardware
Índice 1533: soc.religion.christian
Índice 9589: rec.motorcycles

Documento seleccionado (índice 4113):

What?! White, yes. Well off, definitely not. Hockey season ticket owners have
the lowest average income of any of the four major North American sports.

And think of where the majority of hockey players come from. From a farm out
in Boondock, Saskatchewan or Weedville, Alberta.


The inner city isn't the only place that is poor.
I think the biggest barrier to hockey in the inner city is... no ICE to play on.

						  Stace
 ...

Clases de los 5 documentos más similares:
Índice 9635: rec.sport.hockey
Índice 8180: r

### Analisis de los documentos

**Documento seleccionado (índice 5929):**

En este primer documento a priori, no parece que sea una relación semántica directa, ya que el contenido no coincide exactamente con las categorías de los docs mas similares, por ej. con "Sociedad, Religion y Cristianismo. Tal vez si, con sociedad ya que parece que se están marcando pautas para la comunicación y esa comunicación parece ser de caracter formal. Se interpreta que esto se puede deber al contenido del documento con palabras un poco ambiguas que pueden repetirse en diferentes contextos.

**Documento seleccionado (índice 4113):**

En este segundo documento si se relaciona directamente con las categorías de los documentos seleccionados, ya que habla del deporte de Hockey. Aquí funciona el método utilizado.

**Documento seleccionado (índice 11122):**

En este caso se relaciona bastante con religión, ateismo, y tal vez un poco con política (en términos generales). Se entiende que esto se debe a las palabras claves que representan mas peso por ser relevantes y no estár tan presentes en el resto del texto, por lo que su frecuencia es menor pero con mayor importancia. Aquí también funcionó el algoritmo TF-IDF.

**Documento seleccionado (índice 919):**

Se relaciona directamente.

**Documento seleccionado (índice 7945):**

No se relaciona directamente, el contenido del documento puede ser un poco ambigüo o  no tan definido. Se interpreta que hay palabras claves como: misión, esquema o trupulación que influyeron en relacionarlo con categorías del espacio y deporte.

### 2.

In [21]:
from sklearn.model_selection import GridSearchCV
import itertools
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score

# Definimos los parámetros a probar en el vectorizador
tfidf_params = {
    'ngram_range': [(1, 1), (1, 2)],
    'min_df': [1, 5],
    'max_df': [0.8, 0.95],
}

# Definimos los modelos a evaluar
models = {
    'MultinomialNB': MultinomialNB(),
    'ComplementNB': ComplementNB(),
}

best_score = 0
best_model = None
best_params = {}

# Probamos diferentes combinaciones de parámetros
for vect_params in [dict(zip(tfidf_params, v)) for v in itertools.product(*tfidf_params.values())]:
    vect = TfidfVectorizer(**vect_params)
    X_train_vect = vect.fit_transform(newsgroups_train.data)
    X_test_vect = vect.transform(newsgroups_test.data)

    for model_name, model in models.items():
        model.fit(X_train_vect, y_train)
        y_pred = model.predict(X_test_vect)
        score = f1_score(y_test, y_pred, average='macro')

        if score > best_score:
            best_score = score
            best_model = model_name
            best_params = vect_params

print(f"Mejor modelo: {best_model} con f1-score {best_score}")
print(f"Mejores parámetros: {best_params}")


Mejor modelo: ComplementNB con f1-score 0.692953349950875
Mejores parámetros: {'ngram_range': (1, 1), 'min_df': 1, 'max_df': 0.95}


### 3.

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Transponemos la matriz
X_train_T = X_train.T

# Seleccionamos 5 palabras manualmente
words = ['car', 'computer', 'religion', 'space', 'government']

# Obtenemos todas las palabras en el vocabulario
feature_names = tfidfvect.get_feature_names_out()

for word in words:
    idx = tfidfvect.vocabulary_.get(word)
    if idx is not None:
        cossim = cosine_similarity(X_train_T[idx].reshape(1, -1), X_train_T)[0]
        mostsim = np.argsort(cossim)[::-1][1:6]

        print(f"\nPalabra seleccionada: {word}")
        print("Palabras más similares:")
        for i in mostsim:
            print(feature_names[i])
    else:
        print(f"\nPalabra '{word}' no encontrada en el vocabulario.")



Palabra seleccionada: car
Palabras más similares:
cars
criterium
civic
owner
dealer

Palabra seleccionada: computer
Palabras más similares:
decwriter
harkens
deluged
shopper
the

Palabra seleccionada: religion
Palabras más similares:
religious
religions
categorized
purpsoe
crusades

Palabra seleccionada: space
Palabras más similares:
nasa
seds
shuttle
enfant
seti

Palabra seleccionada: government
Palabras más similares:
the
to
of
libertarian
encryption


### **Análisis**:

En términos generales las palabras están relacionadas con las elegidas, pero hay algunas que no, por ejemplo: criterium y civic con car. Los resultados incluyen palabras comunes y de alta frecuencia y palabras con errores tipográficos purpsoe en lugar de purpose. Para ello se podría preprosesar los datos y hacer un ajuste mas fino en el TfidfVectorizer.